In [ ]:
import sys
import os

sys.path.append('F:/ai-ml/itapia/backend/evo_worker')
sys.path.append('F:/ai-ml/itapia/backend/shared')

In [ ]:
os.environ['POSTGRES_HOST'] = 'localhost'
os.environ['REDIS_HOST'] = 'localhost'

In [ ]:
from app.dependencies import get_backtest_context_manager, create_dependencies
create_dependencies()

In [ ]:
context_mng = get_backtest_context_manager()

In [ ]:
context_mng.init_ready_contexts()

In [ ]:
import nest_asyncio
nest_asyncio.apply()
await context_mng.get_context('AAPL').load_data_into_memory()

In [ ]:
test_ctx = context_mng.get_context('AAPL')

In [ ]:
from app.backtest.evaluator import FitnessEvaluator
from app.backtest.action import ShortSwingActionMapper

In [ ]:
action_mapper = ShortSwingActionMapper()

In [ ]:
evaluator = FitnessEvaluator(test_ctx, action_mapper)

In [ ]:
from itapia_common.dblib.services import RuleService
from itapia_common.dblib.session import get_rdbms_session

db = next(get_rdbms_session())
rule_service = RuleService(db)

In [ ]:
rules = rule_service.get_all_active_rules()

In [ ]:
test_rule_ent = rules[2]

In [ ]:
from itapia_common.rules.rule import Rule

test_rule = Rule.from_entity(test_rule_ent)

In [ ]:
obj, metrics = evaluator.evaluate(test_rule)

In [ ]:
metrics

In [ ]:
obj

In [ ]:
# Trong notebook của bạn, đảm bảo có các import này
from itapia_common.rules.rule import Rule
from itapia_common.rules.parser import parse_tree
from itapia_common.rules import names as nms
from itapia_common.rules.nodes.registry import create_node, get_nodes_by_type
from itapia_common.schemas.enums import NodeType, SemanticType
from itapia_common.schemas.entities.rules import RuleEntity, NodeEntity

In [ ]:
sell_strong_node = create_node(nms.CONST_NUM(-0.8))

In [ ]:
get_nodes_by_type(node_type=NodeType.OPERATOR, purpose=SemanticType.DECISION_SIGNAL)

In [ ]:
from datetime import datetime, timezone
rule_ent = RuleEntity(
    rule_id='SELL_OVER',
    name='sell',
    description='',
    purpose=SemanticType.DECISION_SIGNAL,
    version=1.0,
    is_active=True,
    created_at=datetime.now(timezone.utc),
    updated_at=datetime.now(timezone.utc),
    root=NodeEntity(
        node_name=nms.OPR_TO_DECISION_SIGNAL,
        children=[
            NodeEntity(node_name=nms.CONST_NUM(-0.8))
        ]
    )
)

In [ ]:
test_rule = Rule.from_entity(rule_ent)

In [ ]:
obj, metrics1 = evaluator.evaluate(test_rule)

In [ ]:
from app.backtest.evaluator import metrics

In [ ]:
metrics.trades_df['action_type'].value_counts()